In [1]:
#installation of required packages
!pip install dlib
!pip install face-recognition
!pip install imutils

In [2]:
#blur detection
import cv2
import numpy as np
import glob
from PIL import Image
from matplotlib import pyplot as plt
import face_recognition
folders = glob.glob('../input/hulk-frames/Bruce Banner- Hulk')
imagenames_list = []
for folder in folders:
    for f in glob.glob(folder+'/*.png'):
        imagenames_list.append(f)
laplacian_values=[]
grayimages_names= []        
for image in imagenames_list:
    grayimages_names.append(cv2.imread(image, cv2.IMREAD_GRAYSCALE))
for i in grayimages_names:
 x= cv2.Laplacian(i, cv2.CV_64F).var()
 laplacian_values.append(x)
a=sorted(laplacian_values) 
highest_five_laplacian_values=[]
#print(highest_five_laplacian_values)
for i in range(-1,-6,-1):
    highest_five_laplacian_values.append(laplacian_values.index(a[i]))

In [3]:
#applying histogram equalization
fig = plt.figure(figsize=(20, 20))
rows=5
columns=2
j=0
matrix_array=[]
for i in range(0,5):
    path=cv2.imread(imagenames_list[i],cv2.IMREAD_GRAYSCALE)
    equ=cv2.equalizeHist(path)
    matrix_array.append(equ)
    fig.add_subplot(rows, columns, j+1)
    plt.imshow(path,cmap='gray')
    plt.axis('off')
    plt.title("Before Applying Histogram Equalization")
    fig.add_subplot(rows, columns, j+2)
    plt.imshow(equ,cmap='gray')
    plt.axis('off')
    plt.title("After Applying Histogram Equalization")
    j=j+2

In [4]:
#images extraction after applying histogram equalization
names=['One','Two','Three','Four','Five']
images=[]
for i in range(0,5):
    data=matrix_array[i]
    array=np.array(data,dtype=np.uint8)
    my_image=Image.fromarray(array)
    my_image.save(names[i]+'.png')
    p=cv2.imread(names[i]+'.png')
    images.append(names[i]+'.png')
    figu = plt.figure(figsize=(5, 5))
    rows=1
    columns=1
    figu.add_subplot(rows,columns,1)
    plt.imshow(p)
    plt.xticks([]), plt.yticks([])
#print(images)

In [5]:
#face detection
original_image=cv2.imread('./Four.png')
grayscale_image = cv2.cvtColor(original_image,cv2.COLOR_BGR2GRAY)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
faces=face_cascade.detectMultiScale(grayscale_image,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
for (column, row, width, height) in faces:
    cv2.rectangle(
        original_image,
        (column, row),
        (column + width, row + height),
        (0, 255, 0),
        5
    )
print(faces)
data=original_image
array=np.array(data,dtype=np.uint8)
my_image=Image.fromarray(array)
my_image.save('Face_Detection'+'.png')
p=cv2.imread('Face_Detection'+'.png')
img = cv2.imread('./Face_Detection.png')
plt.imshow(img, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()
print(images)


In [6]:
#face detection
for i in images:
    imgelon =face_recognition.load_image_file(i)
    #print(imgelon)
    imgelon_rgb= cv2.cvtColor(imgelon,cv2.COLOR_BGR2RGB)
    #print(imgelon_rgb[0])
    #----------Finding face Location for drawing bounding boxes-------
    face = face_recognition.face_locations(imgelon_rgb)[0]
    copy = imgelon.copy()
    #-------------------Drawing the Rectangle-------------------------
    print(face)
    cv2.rectangle(copy, (face[3], face[0]),(face[1], face[2]), (255,0,255), 5)
    figu = plt.figure(figsize=(5,5))
    rows=1
    columns=1
    figu.add_subplot(rows,columns,1)
    #plt.imshow(p)
    plt.xticks([]), plt.yticks([])
    plt.imshow(copy)
    plt.xticks([]), plt.yticks([])
    #plt.imshow(imgelon)

In [7]:
#model training
from imutils import paths
import face_recognition
import pickle
import cv2
import os
 
#get paths of each file in folder named Images
#Images here contains my data(folders of various persons)
imagePaths = list(paths.list_images('../input/trainingdata1/Avengers/cropped_images'))
knownEncodings = []
knownNames = []
name=''
# loop over the image paths
#print(imagePaths)
for (i, imagePath) in enumerate(imagePaths):
    # extract the person name from the image path
    if imagePath.split(os.path.sep)[-1][0:8]=='scarlet':
        name='BlackWidow'
    elif imagePath.split(os.path.sep)[-1][0:5]=='chris':
        name='CaptainAmerica'
    elif imagePath.split(os.path.sep)[-1][0:4]=='mark':
        name='Hulk'
    elif imagePath.split(os.path.sep)[-1][0:6]=='robert':
        name='IronMan'
    elif imagePath.split(os.path.sep)[-1][0:5]=='chris':
        name='Thor'
    #print(name)
    # load the input image and convert it from BGR (OpenCV ordering)
    # to dlib ordering (RGB)
    image = cv2.imread(imagePath)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #Use Face_recognition to locate faces
    boxes = face_recognition.face_locations(rgb,model='hog')
    # compute the facial embedding for the face
    encodings = face_recognition.face_encodings(rgb, boxes)
    # loop over the encodings
    for encoding in encodings:
        knownEncodings.append(encoding)
        knownNames.append(name)
#save emcodings along with their names in dictionary data
data = {"encodings": knownEncodings, "names": knownNames}
#use pickle to save data into a file for later use
f = open("face_enc", "wb")
f.write(pickle.dumps(data))
f.close()
#print(data)

In [18]:
names=[]
for i in images:
    image=cv2.imread(i)
    rgb=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    encodings = face_recognition.face_encodings(rgb)
    # loop over the facial embeddings incase
    # we have multiple embeddings for multiple faces
    for encoding in encodings:
        #Compare encodings with encodings in data["encodings"]
        #Matches contain array with boolean values and True for the embeddings it matches closely
        #and False for rest
        matches = face_recognition.compare_faces(data["encodings"],encoding)
        #set name =inknown if no encoding matches
        #name = "Unknown"
        # check to see if we have found a match
        unknown=["Unknown"]
        if True in matches:
            #Find positions at which we get True and store them
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}
            # loop over the matched indexes and maintain a count for
            # each recognized face
            for i in matchedIdxs:
                #Check the names at respective indexes we stored in matchedIdxs
                name = data["names"][i]
                #increase count for the name we got
                counts[name] = counts.get(name, 0) + 1
                #set name which has highest count
                name = max(counts, key=counts.get)
                #print(name)
 
            # update the list of names
                names.append(name)
                #print(names)
                #print(face)
            # loop over the recognized faces
            for((x,y,w,h),name) in zip(faces,names):
                # rescale the face coordinates
                # draw the predicted face name on the image
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0),2)
                cv2.putText(image,name, (x,y), cv2.FONT_HERSHEY_SIMPLEX,
                 1, (0, 255, 0),3)
        else:
            names.append("Unknown")
            # loop over the recognized faces
            for ((x, y, w, h), name) in zip(faces,unknown):
                # rescale the face coordinates
                # draw the predicted face name on the image
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0),2)
                cv2.putText(image, name, (x,y), cv2.FONT_HERSHEY_SIMPLEX,
                 1, (0, 255, 0),3)
        #print(names)
        figu = plt.figure(figsize=(5,5))
        rows=1
        columns=1
        figu.add_subplot(rows,columns,1)
        plt.imshow(p)
        #plt.xticks([]), plt.yticks([])
        #plt.imshow(copy)
        #plt.xticks([]), plt.yticks([])
        plt.imshow(image)
        plt.xticks([]), plt.yticks([])

In [17]:
def names_count(images_list):
    counts = dict()
    for image in images_list:
        if image in counts:
            counts[image] += 1
        else:
            counts[image] = 1
    counts_x = sorted(counts.items(), key=lambda kv: kv[1])
    #print(counts_x)
    return counts_x
print(len(images))
#print(names)
highest=names_count(images)
#print(highest)
#print(highest[0])

In [10]:
Avenger_Names=['CaptainAmerica','IronMan','Hulk','Thor','BlackWidow']
if highest[0] in Avenger_Names:
    print('Is an Avenger')
else:
    print('Is Not an Avenger')